# Ensemble Methods - Proje: Kapsamlı Model Karşılaştırması

Bu proje, farklı ensemble yöntemlerini karşılaştırır ve en iyi performansı gösterir.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import VotingClassifier, BaggingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.datasets import make_classification

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


In [ ]:
# Veri seti
X, y = make_classification(n_samples=2000, n_features=20, n_informative=15, 
                           n_redundant=5, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Base modeller
base_models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Ensemble modeller
voting = VotingClassifier(
    estimators=list(base_models.items())[:3],
    voting='soft'
)

bagging = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=50,
    random_state=42
)

stacking = StackingClassifier(
    estimators=list(base_models.items())[:3],
    final_estimator=LogisticRegression(random_state=42),
    cv=5
)

# Tüm modelleri eğit ve değerlendir
results = {}

# Base modeller
for name, model in base_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    results[name] = {
        'Test Accuracy': accuracy_score(y_test, y_pred),
        'CV Accuracy Mean': cv_scores.mean(),
        'CV Accuracy Std': cv_scores.std()
    }

# Ensemble modeller
voting.fit(X_train, y_train)
bagging.fit(X_train, y_train)
stacking.fit(X_train, y_train)

results['Voting Classifier'] = {
    'Test Accuracy': accuracy_score(y_test, voting.predict(X_test)),
    'CV Accuracy Mean': cross_val_score(voting, X_train, y_train, cv=5, scoring='accuracy').mean(),
    'CV Accuracy Std': cross_val_score(voting, X_train, y_train, cv=5, scoring='accuracy').std()
}

results['Bagging Classifier'] = {
    'Test Accuracy': accuracy_score(y_test, bagging.predict(X_test)),
    'CV Accuracy Mean': cross_val_score(bagging, X_train, y_train, cv=5, scoring='accuracy').mean(),
    'CV Accuracy Std': cross_val_score(bagging, X_train, y_train, cv=5, scoring='accuracy').std()
}

results['Stacking Classifier'] = {
    'Test Accuracy': accuracy_score(y_test, stacking.predict(X_test)),
    'CV Accuracy Mean': cross_val_score(stacking, X_train, y_train, cv=5, scoring='accuracy').mean(),
    'CV Accuracy Std': cross_val_score(stacking, X_train, y_train, cv=5, scoring='accuracy').std()
}

# Sonuçları göster
results_df = pd.DataFrame(results).T
print("Kapsamlı Model Karşılaştırması:")
print(results_df.round(4))

# Görselleştir
plt.figure(figsize=(12, 6))
results_df['Test Accuracy'].sort_values(ascending=True).plot(kind='barh')
plt.xlabel('Test Accuracy')
plt.title('Model Performans Karşılaştırması')
plt.tight_layout()
plt.show()
